In [ ]:
import pandas as pd
import numpy as np
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u

# --- 1. Cosmological Model Setup ---
# Define the Flat Lambda-CDM model
# H0 = 70 km/s/Mpc, Om0 = 0.3
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
print("Cosmological model (Flat Lambda-CDM): H0=70, Om0=0.3")

# --- 2. Data Loading and Preparation ---
INPUT_FILE = 'des_good_coverage.csv'
OUTPUT_FILE = 'des_test_data.csv'
MAG_COLUMNS = ['g_pk_mag', 'r_pk_mag', 'i_pk_mag', 'z_pk_mag']
REDSHIFT_COLUMN = 'photo_z'

try:
    # Read the full DataFrame
    df = pd.read_csv(INPUT_FILE)
except FileNotFoundError:
    print(f"Error: The file '{INPUT_FILE}' was not found. Please check the file path.")
    exit()




print(f"Total rows read: {len(df)}")

# Extract data arrays
apparent_mags = df[MAG_COLUMNS].to_numpy()
redshifts = df[REDSHIFT_COLUMN].to_numpy()
NUM_MAG_BANDS = apparent_mags.shape[1]

# Initialize the output array for absolute magnitudes
absolute_mags = np.zeros_like(apparent_mags, dtype=float)



# Loop through each object (row)
for i in range(len(df)):
    z = redshifts[i]

    # Handle non-physical or zero redshifts
    if z <= 0 or not np.isfinite(z):
        z=0.028
        
    
    # Calculate luminosity distance (d)
    # The result 'ld' is in astropy units (Mpc)
    ld = cosmo.luminosity_distance(z)

    # Convert to parsecs (pc) and get the value (d is required in pc for the formula)
    d_pc = ld.to(u.pc).value

   

  
    distance_modulus = 5.0 - 5.0 * np.log10(d_pc)

    # Apply the calculation to all magnitude bands for this object
    absolute_mags[i, :] = apparent_mags[i, :] + distance_modulus



# --- 4. Data Replacement and Saving ---
# Create new column names for absolute magnitudes
NEW_MAG_COLUMNS = MAG_COLUMNS # e.g., 'M_g', 'M_r'

# Replace the apparent magnitude columns with the calculated absolute magnitude columns
# Note: We are doing this *in place* on a copy of the original DataFrame's columns
df[NEW_MAG_COLUMNS] = absolute_mags

# Drop the original apparent magnitude columns if you want to replace them completely
df = df.drop(columns=MAG_COLUMNS)

# Save the resulting DataFrame to a new CSV file
df.to_csv(OUTPUT_FILE, index=False)

print(f"\nSuccess! Results saved to '{OUTPUT_FILE}'.")
print(f"DataFrame now contains absolute magnitude columns: {NEW_MAG_COLUMNS}")

Cosmological model (Flat Lambda-CDM): H0=70, Om0=0.3
Total rows read: 456

Success! Results saved to 'des_good_absolute_mags.csv'.
DataFrame now contains absolute magnitude columns: ['g_pk_mag', 'r_pk_mag', 'i_pk_mag', 'z_pk_mag']
